# Discrete Choice Models Overview

In [1]:
import numpy as np
import statsmodels.api as sm

## Data

Load data from Spector and Mazzeo (1980). Examples follow Greene's Econometric Analysis Ch. 21 (5th Edition).

In [29]:
spector_data = sm.datasets.spector.load_pandas().data


In [30]:
spector_data.head()

,GPA,TUCE,PSI,GRADE
0,2.66,20.0,0.0,0.0
1,2.89,22.0,0.0,0.0
2,3.28,24.0,0.0,0.0
3,2.92,12.0,0.0,0.0
4,4.00,21.0,0.0,1.0


In [31]:
spector_data.exog = sm.add_constant(spector_data.exog, prepend=False)

AttributeError: ignored

Inspect the data:

In [26]:
print(spector_data.exog[:10])
print(spector_data.endog[:10])

[[ 2.66 20.    0.    1.  ]
 [ 2.89 22.    0.    1.  ]
 [ 3.28 24.    0.    1.  ]
 [ 2.92 12.    0.    1.  ]
 [ 4.   21.    0.    1.  ]
 [ 2.86 17.    0.    1.  ]
 [ 2.76 17.    0.    1.  ]
 [ 2.87 21.    0.    1.  ]
 [ 3.03 25.    0.    1.  ]
 [ 3.92 29.    0.    1.  ]]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]


## Linear Probability Model (OLS)

In [4]:
lpm_mod = sm.OLS(spector_data.endog, spector_data.exog)
lpm_res = lpm_mod.fit()
print("Parameters: ", lpm_res.params[:-1])

Parameters:  [0.46385168 0.01049512 0.37855479]


## Logit Model

In [5]:
logit_mod = sm.Logit(spector_data.endog, spector_data.exog)
logit_res = logit_mod.fit(disp=0)
print("Parameters: ", logit_res.params)

Parameters:  [  2.82611259   0.09515766   2.37868766 -13.02134686]


Marginal Effects

In [6]:
margeff = logit_res.get_margeff()
print(margeff.summary())

        Logit Marginal Effects       
Dep. Variable:                      y
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3626      0.109      3.313      0.001       0.148       0.577
x2             0.0122      0.018      0.686      0.493      -0.023       0.047
x3             0.3052      0.092      3.304      0.001       0.124       0.486


As in all the discrete data models presented below, we can print a nice summary of results:

In [7]:
print(logit_res.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   32
Model:                          Logit   Df Residuals:                       28
Method:                           MLE   Df Model:                            3
Date:                Tue, 25 Oct 2022   Pseudo R-squ.:                  0.3740
Time:                        08:00:48   Log-Likelihood:                -12.890
converged:                       True   LL-Null:                       -20.592
Covariance Type:            nonrobust   LLR p-value:                  0.001502
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.8261      1.263      2.238      0.025       0.351       5.301
x2             0.0952      0.142      0.672      0.501      -0.182       0.373
x3             2.3787      1.065      2.234      0.0

## Probit Model 

In [8]:
probit_mod = sm.Probit(spector_data.endog, spector_data.exog)
probit_res = probit_mod.fit()
probit_margeff = probit_res.get_margeff()
print("Parameters: ", probit_res.params)
print("Marginal effects: ")
print(probit_margeff.summary())

Optimization terminated successfully.
         Current function value: 0.400588
         Iterations 6
Parameters:  [ 1.62581004  0.05172895  1.42633234 -7.45231965]
Marginal effects: 
       Probit Marginal Effects       
Dep. Variable:                      y
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3608      0.113      3.182      0.001       0.139       0.583
x2             0.0115      0.018      0.624      0.533      -0.025       0.048
x3             0.3165      0.090      3.508      0.000       0.140       0.493


## Multinomial Logit

Load data from the American National Election Studies:

In [32]:
dta = sm.datasets.anes96.load_pandas().data

In [33]:
dta.head()

,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,educ,income,vote,logpopul
0,0.0,7.0,7.0,1.0,6.0,6.0,36.0,3.0,1.0,1.0,-2.302585
1,190.0,1.0,3.0,3.0,5.0,1.0,20.0,4.0,1.0,0.0,5.247550
2,31.0,7.0,2.0,2.0,6.0,1.0,24.0,6.0,1.0,0.0,3.437208
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624


In [34]:
anes_data = sm.datasets.anes96.load_pandas()
anes_exog = anes_data.exog
anes_exog = sm.add_constant(anes_exog)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Inspect the data:

In [38]:
print(anes_data.exog.head(100))

    logpopul  selfLR   age  educ  income
0  -2.302585     7.0  36.0   3.0     1.0
1   5.247550     3.0  20.0   4.0     1.0
2   3.437208     2.0  24.0   6.0     1.0
3   4.420045     3.0  28.0   6.0     1.0
4   6.461624     5.0  68.0   6.0     1.0
..       ...     ...   ...   ...     ...
95  3.437208     4.0  53.0   3.0     6.0
96  2.778819     5.0  82.0   3.0     6.0
97  3.499533     4.0  82.0   3.0     6.0
98 -2.302585     5.0  47.0   6.0     7.0
99 -2.302585     4.0  68.0   3.0     7.0

[100 rows x 5 columns]


In [39]:
print(anes_data.endog.head(100))

0     6.0
1     1.0
2     1.0
3     1.0
4     0.0
     ... 
95    1.0
96    5.0
97    6.0
98    5.0
99    4.0
Name: PID, Length: 100, dtype: float64


Fit MNL model:

In [22]:
mlogit_mod = sm.MNLogit(anes_data.endog, anes_exog)
mlogit_res = mlogit_mod.fit()
print(mlogit_res.params)

Optimization terminated successfully.
         Current function value: 1.548647
         Iterations 7
[[-3.73401677e-01 -2.25091318e+00 -3.66558353e+00 -7.61384309e+00
  -7.06047825e+00 -1.21057509e+01]
 [-1.15359746e-02 -8.87506530e-02 -1.05966699e-01 -9.15567017e-02
  -9.32846040e-02 -1.40880692e-01]
 [ 2.97714352e-01  3.91668642e-01  5.73450508e-01  1.27877179e+00
   1.34696165e+00  2.07008014e+00]
 [-2.49449954e-02 -2.28978371e-02 -1.48512069e-02 -8.68134503e-03
  -1.79040689e-02 -9.43264870e-03]
 [ 8.24914421e-02  1.81042758e-01 -7.15241904e-03  1.99827955e-01
   2.16938850e-01  3.21925702e-01]
 [ 5.19655317e-03  4.78739761e-02  5.75751595e-02  8.44983753e-02
   8.09584122e-02  1.08894083e-01]]


In [24]:
mlogit_margeff = mlogit_res.get_margeff()
print("Parameters: ", mlogit_res.params)
print("Marginal effects: ")
print(mlogit_margeff.summary())

Parameters:  [[-3.73401677e-01 -2.25091318e+00 -3.66558353e+00 -7.61384309e+00
  -7.06047825e+00 -1.21057509e+01]
 [-1.15359746e-02 -8.87506530e-02 -1.05966699e-01 -9.15567017e-02
  -9.32846040e-02 -1.40880692e-01]
 [ 2.97714352e-01  3.91668642e-01  5.73450508e-01  1.27877179e+00
   1.34696165e+00  2.07008014e+00]
 [-2.49449954e-02 -2.28978371e-02 -1.48512069e-02 -8.68134503e-03
  -1.79040689e-02 -9.43264870e-03]
 [ 8.24914421e-02  1.81042758e-01 -7.15241904e-03  1.99827955e-01
   2.16938850e-01  3.21925702e-01]
 [ 5.19655317e-03  4.78739761e-02  5.75751595e-02  8.44983753e-02
   8.09584122e-02  1.08894083e-01]]
Marginal effects: 
       MNLogit Marginal Effects      
Dep. Variable:                      y
Method:                          dydx
At:                           overall
       y=0      dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0087      0.004      2.250      0.

## Poisson

Load the Rand data. Note that this example is similar to Cameron and Trivedi's `Microeconometrics` Table 20.5, but it is slightly different because of minor changes in the data. 

In [ ]:
rand_data = sm.datasets.randhie.load()
rand_exog = rand_data.exog
rand_exog = sm.add_constant(rand_exog, prepend=False)

Fit Poisson model: 

In [ ]:
poisson_mod = sm.Poisson(rand_data.endog, rand_exog)
poisson_res = poisson_mod.fit(method="newton")
print(poisson_res.summary())

## Negative Binomial

The negative binomial model gives slightly different results. 

In [ ]:
mod_nbin = sm.NegativeBinomial(rand_data.endog, rand_exog)
res_nbin = mod_nbin.fit(disp=False)
print(res_nbin.summary())

## Alternative solvers

The default method for fitting discrete data MLE models is Newton-Raphson. You can use other solvers by using the ``method`` argument: 

In [25]:
mlogit_res = mlogit_mod.fit(method="bfgs", maxiter=250)
print(mlogit_res.summary())

Optimization terminated successfully.
         Current function value: 1.548647
         Iterations: 111
         Function evaluations: 117
         Gradient evaluations: 117
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                  944
Model:                        MNLogit   Df Residuals:                      908
Method:                           MLE   Df Model:                           30
Date:                Tue, 25 Oct 2022   Pseudo R-squ.:                  0.1648
Time:                        08:10:34   Log-Likelihood:                -1461.9
converged:                       True   LL-Null:                       -1750.3
Covariance Type:            nonrobust   LLR p-value:                1.822e-102
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3734      0.630   